In [2]:

import codecs
from numpy import *
import matplotlib.pyplot as plt

def plot_cluster(data_mat, cluster_assment, centroid):
	"""
	@brief      plot cluster and centroid
	@param      data_mat        The data matrix
	@param      cluster_assment  The cluste assment
	@param      centroid        The centroid
	@return     
	"""
	plt.figure(figsize=(15, 6), dpi=80)
	plt.subplot(121)
	plt.plot(data_mat[:, 0], data_mat[:, 1], 'o')
	plt.title("source data", fontsize=15)
	plt.subplot(122)
	k = shape(centroid)[0]
	colors = [plt.cm.Spectral(each) for each in linspace(0, 1, k)]
	for i, col in zip(range(k), colors):
	    per_data_set = data_mat[nonzero(cluster_assment[:,0].A == i)[0]]
	    plt.plot(per_data_set[:, 0], per_data_set[:, 1], 'o', markerfacecolor=tuple(col),
	             markeredgecolor='k', markersize=10)
	for i in range(k):
		plt.plot(centroid[:,0], centroid[:,1], '+', color = 'k', markersize=18)
	plt.title("bi_KMeans Cluster, k = 3", fontsize=15)
	plt.show()


In [9]:
def fileInput(filename, columnNum):
    """
    @brief      read data from file
    @param      filename
    @param      columnNum: the range of columns to be read, start from 1
    @return     points_mat: [[position1], [position2], [position3], ... [positionN]]
    """   
    points_set = []

    with codecs.open(filename) as f:
        for line in f.readlines():
            linetmp = line.strip().split()[0:columnNum]
            #! map return a map object, 
            #! so we need to convert it to list or tuple by list(map()) or tuple(map())
            flisttmp = list(map(float, linetmp))
            points_set.append(flisttmp) 
    
    points_mat = mat(points_set)

    return points_mat


def centroid_gen(points_mat, k):
    """
    @brief      generate k centroids
    @param      points_set
    @param      k:          the number of centroids
    @return     centroids:  [[position1], [position2], [position3], ... [positionN]]
    """
    columnNum = shape(points_mat)[1]
    centroids = mat(zeros((k, columnNum)))

    for j in range(columnNum):
        mintmp = min(points_mat[:, j])
        maxtmp = max(points_mat[:, j])
        rangetmp = float(maxtmp - mintmp)
        centroids[:,j] = mat(mintmp + rangetmp * random.rand(k, 1))

    return centroids

def distance_Euclidean(vecA, vecB):
    """
    @brief      calculate the Euclidean distance between two vectors
    @param      vecA
    @param      vecB
    @return     the Euclidean distance
    """
    return sqrt(sum(power(vecA - vecB, 2)))

def k_means(points_mat, k, distance_calculation):
    """
    @brief      kMeans algorithm
    @param      points_mat:             The data matrix
    @param      k:                      num of cluster
    @param      distance_calculation:   The distance funtion
    @return     centroids
    @return     cluster_attribute:      [[cluster_index, distance**2], ...]
    """

    pointsNum = shape(points_mat)[0]

    cluster_attribute = mat(zeros((pointsNum, 2)))  # cluster index, distance

    #* Initialize the centroids

    cluster_num_change = True

    while cluster_num_change:
        cluster_num_change = False

        centroids = centroid_gen(points_mat, k)


        cluster_changed = True

        #* loop until the cluster is not changed
        while cluster_changed:
            cluster_changed = False

            #* assign each point to the nearest centroid
            for i in range(pointsNum):
                cluster_index = None
                min_dist = inf
                for j in range(k):
                    dist_tmp = eval(distance_calculation)(points_mat[i, :], centroids[j, :])
                    if dist_tmp < min_dist:
                        min_dist = dist_tmp
                        cluster_index = j
                
                #* judge if the cluster is changed
                if cluster_attribute[i, 0] != cluster_index:
                    cluster_changed = True

                cluster_attribute[i, :] = cluster_index, min_dist**2

            #* update the centroids, in the new clusters
            for j in range(k):
                pointstmp = points_mat[nonzero(cluster_attribute[:, 0] == j)[0]]
                centroids[j, :] = mean(pointstmp, axis = 0)

            #* check if the total number of clusters is less than k

            #!cluster_attribute[:,0].T.tolist()-> [[]]
            if len(set(cluster_attribute[:,0].T.tolist()[0])) != k:
                cluster_num_change = True

    return centroids, cluster_attribute

#! there is the case
#! that all the points are in one cluster
def bi_kmeans(points_mat, k, distance_calculation):

    centroids_bi = []
    pointsNum = shape(points_mat)[0]
    cluster_attribute = mat(zeros((pointsNum, 2)))  # cluster index, distance

    #* Initialize the centroids
    #! at first, only one centroid
    #! here the output of mean is a matrix, with two shells [[x,x]]
    centroids_bi.append(mean(points_mat, axis = 0).tolist()[0])

    #* Initialize the cluster attribute
    for i in range(pointsNum):
        cluster_attribute[i,1] = eval(distance_calculation)(centroids_bi[0], points_mat[i, :]) ** 2

    while len(centroids_bi) < k:
        lowest_sse = inf
        for i in range(len(centroids_bi)):
            points_mat_tmp = points_mat[nonzero(cluster_attribute[:, 0] == i)[0],:]
            #* there are two clusters, index0 and index1
            centroids_bi_tmp, cluster_attribute_tmp = k_means(points_mat_tmp, 2, distance_calculation)
            sse_split = sum(cluster_attribute_tmp[:, 1])
            #! only calculate in the new cluster
            sse_nonsplit = sum(cluster_attribute[nonzero(cluster_attribute[:, 0] != i)[0], 1])

            #* find the BEST split in all cluster and ONLY choose one in each loop
            if sse_split + sse_nonsplit < lowest_sse:
                best_split_index = i
                lowest_sse = sse_split + sse_nonsplit
                best_cluster_attribute = cluster_attribute_tmp.copy()
                best_centroids_bi = centroids_bi_tmp.copy()

        #! for the split of cluster i
        #! i -> index 0, and len -> index 1 
        
        best_cluster_attribute[nonzero(best_cluster_attribute[:,0]==1)[0],0] = len(centroids_bi)
        best_cluster_attribute[nonzero(best_cluster_attribute[:,0]==0)[0],0] = best_split_index
        
        cluster_attribute[nonzero(cluster_attribute[:,0]==best_split_index)[0],:] = best_cluster_attribute

        centroids_bi[best_split_index] = best_centroids_bi[0,:].tolist()[0]
        centroids_bi.append(best_centroids_bi[1,:].tolist()[0])

    
    centroids_bi = mat(centroids_bi)
    
    return centroids_bi, cluster_attribute
                

                
            
    
            

        


distance_calculation = "distance_Euclidean"
filename = "attractors.txt"
columnNum = 3
k = 14

points_mat = fileInput(filename, columnNum)

centroids_bi, cluster_attribute = bi_kmeans(points_mat, k, distance_calculation)










In [57]:
attractors = list(set(cluster_attribute[:,0].T.tolist()[0]))

for i in range(len(attractors)):
    temp = nonzero(cluster_attribute[:,0] == attractors[i])[0].tolist()
    temp = [x+1 for x in temp]
    print(f"cluster: {attractors[i]}, len: {len(temp)}, attractors: {temp}")

cluster: 0.0, len: 11, attractors: [109, 119, 123, 126, 130, 131, 138, 142, 144, 151, 152]
cluster: 1.0, len: 16, attractors: [2, 15, 16, 44, 48, 53, 61, 69, 70, 77, 80, 84, 89, 101, 104, 108]
cluster: 2.0, len: 7, attractors: [79, 87, 111, 117, 118, 120, 149]
cluster: 3.0, len: 11, attractors: [74, 92, 94, 110, 128, 134, 140, 143, 148, 153, 154]
cluster: 4.0, len: 8, attractors: [7, 13, 18, 23, 25, 29, 30, 34]
cluster: 5.0, len: 14, attractors: [37, 40, 42, 55, 59, 62, 63, 65, 68, 72, 81, 88, 93, 95]
cluster: 6.0, len: 11, attractors: [39, 41, 43, 47, 57, 60, 64, 71, 82, 83, 96]
cluster: 7.0, len: 12, attractors: [105, 121, 122, 124, 129, 132, 133, 136, 137, 141, 150, 156]
cluster: 8.0, len: 13, attractors: [1, 4, 5, 10, 12, 19, 22, 24, 26, 27, 28, 32, 33]
cluster: 9.0, len: 11, attractors: [76, 90, 97, 103, 112, 113, 114, 115, 116, 125, 145]
cluster: 10.0, len: 14, attractors: [17, 38, 49, 50, 51, 52, 66, 67, 98, 99, 100, 102, 106, 107]
cluster: 11.0, len: 12, attractors: [9, 20, 21,